In [3]:
pos_max = {"C": 1, "1B": 1, "2B": 1, "SS": 1, "3B": 1, "1B/3B":1, "2B/SS":1, "CF":2, "LF":2, "RF":2, 'P':10}


In [173]:
import pandas as pd
import numpy as np

class DraftState:
    def __init__(self, rosters, turns, freeagents, playerjm=None):
        self.rosters = rosters
        self.freeagents = freeagents
        self.turns = turns
        self.playerJustMoved = playerjm

class MLBPlayer:
    def __init__(self, name, team, position, points):
        self.name = name
        self.team = team
        self.position = str(position)
        self.points = points
    def __repr__(self):
        return self.name

def GetResult(self, playerjm):
    """ Get the game result from the viewpoint of playerjm.
    """
    if playerjm is None: return 0
    
    pos_wgts = {
        ("C"): [1],
        ("1B"): [1],
        ("2B"): [1],
        ("SS"): [1],
        ('3B'): [1],
        # ("1B", "3B"): [.6, .4],
        # ("2B", "SS"): [.6, .4],
        # ('LF'):[.3],
        # ('CF'):[.3],
        ('OF'):[1],
        ("C", "1B", "2B", "SS", "3B", 'DH'): [1],
        ("SP"): [1],
        ("RP"): [1]
    }
    result = 0
    # map the drafted players to the weights
    for p in self.rosters[playerjm]:
        max_wgt, _, max_pos, old_wgts = max(
            ((wgts[0], -len(lineup_pos), lineup_pos, wgts) for lineup_pos, wgts in pos_wgts.items()
                if p.position in lineup_pos),
            default=(0, 0, (), []))
        if max_wgt > 0:
            result += max_wgt * p.points
            old_wgts.pop(0)
            if not old_wgts:
                pos_wgts.pop(max_pos)
                
    # map the remaining weights to the top three free agents
    for pos, wgts in pos_wgts.items():
        result += np.mean([p.points for p in self.freeagents if p.position in pos][:3]) * sum(wgts)
    return result
DraftState.GetResult = GetResult

def GetMoves(self):
    """ Get all possible moves from this state.
    """
    
    pos_max = {"C": 1, "1B": 2, "2B": 2, "SS": 2, "3B": 2, "OF":6, 'P':10, 'DH':1}
    if len(self.turns) == 0: return []
    roster_positions = np.array([p.position for p in  self.rosters[self.turns[0]]], dtype=str)
    moves = [pos for pos, max_ in pos_max.items() if np.sum(roster_positions == pos) < max_]
    return moves
DraftState.GetMoves = GetMoves

def DoMove(self, move):
    """ Update a state by carrying out the given move.
        Must update playerJustMoved.
    """
    player = next(p for p in self.freeagents if p.position == move)
    self.freeagents.remove(player)
    rosterId = self.turns.pop(0)
    self.rosters[rosterId].append(player)
    self.playerJustMoved = rosterId
    
DraftState.DoMove = DoMove

def Clone(self):
    """ Create a deep clone of this game state.
    """
    rosters = list(map(lambda r: r[:], self.rosters))
    st = DraftState(rosters, self.turns[:], self.freeagents[:],
            self.playerJustMoved)
    return st
DraftState.Clone = Clone

# This is a very simple implementation of the UCT Monte Carlo Tree Search algorithm in Python 2.7.
# The function UCT(rootstate, itermax, verbose = False) is towards the bottom of the code.
# It aims to have the clearest and simplest possible code, and for the sake of clarity, the code
# is orders of magnitude less efficient than it could be made, particularly by using a 
# state.GetRandomMove() or state.DoRandomRollout() function.
# 
# Written by Peter Cowling, Ed Powley, Daniel Whitehouse (University of York, UK) September 2012.
# 
# Licence is granted to freely use and distribute for any sensible/legal purpose so long as this comment
# remains in any distributed code.
# 
# For more information about Monte Carlo Tree Search check out our web site at www.mcts.ai
from math import *
import random
class Node:
    """ A node in the game tree. Note wins is always from the viewpoint of playerJustMoved.
        Crashes if state not specified.
    """
    def __init__(self, move = None, parent = None, state = None):
        self.move = move # the move that got us to this node - "None" for the root node
        self.parentNode = parent # "None" for the root node
        self.childNodes = []
        self.wins = 0
        self.visits = 0
        self.untriedMoves = state.GetMoves() # future child nodes
        self.playerJustMoved = state.playerJustMoved # the only part of the state that the Node needs later
        
    def UCTSelectChild(self):
        """ Use the UCB1 formula to select a child node. Often a constant UCTK is applied so we have
            lambda c: c.wins/c.visits + UCTK * sqrt(2*log(self.visits)/c.visits to vary the amount of
            exploration versus exploitation.
        """
        UCTK = 200
        s = sorted(self.childNodes, key = lambda c: c.wins/c.visits + UCTK * sqrt(2*log(self.visits)/c.visits))[-1]
        return s
    
    def AddChild(self, m, s):
        """ Remove m from untriedMoves and add a new child node for this move.
            Return the added child node
        """
        n = Node(move = m, parent = self, state = s)
        self.untriedMoves.remove(m)
        self.childNodes.append(n)
        return n
    
    def Update(self, result):
        """ Update this node - one additional visit and result additional wins. result must be from the viewpoint of playerJustmoved.
        """
        self.visits += 1
        self.wins += result
def UCT(rootstate, itermax, verbose = False):
    """ Conduct a UCT search for itermax iterations starting from rootstate.
        Return the best move from the rootstate.
    """
    rootnode = Node(state = rootstate)
    for i in range(itermax):
        node = rootnode
        state = rootstate.Clone()
        # Select
        while node.untriedMoves == [] and node.childNodes != []: # node is fully expanded and non-terminal
            node = node.UCTSelectChild()
            state.DoMove(node.move)
        # Expand
        if node.untriedMoves != []: # if we can expand (i.e. state/node is non-terminal)
            m = random.choice(node.untriedMoves) 
            state.DoMove(m)
            node = node.AddChild(m,state) # add child and descend tree
        # Rollout - this can often be made orders of magnitude quicker using a state.GetRandomMove() function
        while state.GetMoves() != []: # while state is non-terminal
            state.DoMove(random.choice(state.GetMoves()))
        # Backpropagate
        while node != None: # backpropagate from the expanded node and work back to the root node
            node.Update(state.GetResult(node.playerJustMoved)) # state is terminal. Update node with result from POV of node.playerJustMoved
            node = node.parentNode
    return sorted(rootnode.childNodes, key = lambda c: c.visits)[-1].move # return the move that was most visited

In [205]:
mlb_players = pd.read_csv("./R-stuff/players.csv", index_col=0)



In [210]:
freeagents = [MLBPlayer(*p) for p in mlb_players.itertuples(index=False, name=None)]


In [215]:
freeagents[*Will]

SyntaxError: invalid syntax (1762798384.py, line 1)

In [193]:
mlb_players = pd.read_csv("./R-stuff/players.csv", index_col=0)

freeagents = [MLBPlayer(*p) for p in mlb_players.itertuples(index=False, name=None)]

num_competitors = 10
rosters = [[] for _ in range(num_competitors)] # empty rosters to start with

# Add keepers

# These are the list numbers of the players for each manager
Korn = [6, 15, 49, 311] # Alonso, Tatis, Franco, Sandy Alc
Alden = [17, 314, 319, 29] # Olson, Gausman, Darvish, Abreu
Kerry = [26, 28, 64, 320] # Reynolds, Bregman, JT Realmuto, Cease
Alex = [9, 13, 16, 22] # Devers, Machado, Acuna, Schwarber
Brent = [14, 18, 19, ] # Ohtani, Rodriguez, Seager
Dakota = [3, 11, 41, 302] # Trout, JoRam, Semien, Burnes
Jake = [2, 4, 8, 94] # Vlad Jr, Soto, Riley, Harper
John = [23, 25, 36, 46] # Arenado, Eloy, Xander, Lindor
Will = [1, 5, 10, 12] # Betts, Freeman, Alvarez, Tucker
Cuyler = [0, 7, 300, 301] # Judge, Goldy, Degrom, Cole
teams = [Korn, Alden, Kerry, Alex, Brent, Dakota, Jake, John, Will, Cuyler]
rosters[0] = [freeagents[i] for i in Korn]
rosters[1] = [freeagents[i] for i in Alden]
rosters[2] = [freeagents[i] for i in Kerry]
rosters[3] = [freeagents[i] for i in Alex]
rosters[4] = [freeagents[i] for i in Brent]
rosters[5] = [freeagents[i] for i in Dakota]
rosters[6] = [freeagents[i] for i in Jake]
rosters[7] = [freeagents[i] for i in John]
rosters[8] = [freeagents[i] for i in Will]
rosters[9] = [freeagents[i] for i in Cuyler]

In [178]:
num_rounds = 10
turns = []
# generate turns by snake order
for i in range(num_rounds):
    turns += reversed(range(num_competitors)) if i % 2 else range(num_competitors)
    
state = DraftState(rosters, turns, freeagents)
iterations = 1000
while state.GetMoves() != []:
    move = UCT(state, iterations)
    print(move, end=".")
    state.DoMove(move)

OF.OF.DH.1B.1B.3B.P.1B.1B.3B.3B.3B.2B.SS.SS.3B.SS.SS.3B.C.2B.C.1B.P.P.P.P.P.C.2B.SS.2B.P.C.1B.2B.P.P.SS.3B.P.2B.2B.2B.C.C.2B.C.SS.C.DH.P.3B.P.SS.P.SS.2B.OF.DH.OF.P.P.1B.P.DH.1B.SS.P.1B.OF.SS.OF.OF.2B.3B.DH.SS.2B.3B.2B.3B.3B.2B.SS.OF.P.OF.3B.OF.OF.OF.OF.P.OF.2B.C.OF.DH.P.

In [179]:
pd.DataFrame({"Team " + str(i + 1): r for i, r in enumerate(state.rosters)})

ValueError: All arrays must be of the same length

In [183]:
tm1 = pd.DataFrame([r.name for i, r in enumerate(state.rosters[0])])
tm2 = pd.DataFrame([r.name for i, r in enumerate(state.rosters[1])])
tm3 = pd.DataFrame([r.name for i, r in enumerate(state.rosters[2])])
tm4 = pd.DataFrame([r.name for i, r in enumerate(state.rosters[3])])
tm5 = pd.DataFrame([r.name for i, r in enumerate(state.rosters[4])])
tm6 = pd.DataFrame([r.name for i, r in enumerate(state.rosters[5])])
tm7 = pd.DataFrame([r.name for i, r in enumerate(state.rosters[6])])
tm8 = pd.DataFrame([r.name for i, r in enumerate(state.rosters[7])])
tm9 = pd.DataFrame([r.name for i, r in enumerate(state.rosters[8])])
tm10 = pd.DataFrame([r.name for i, r in enumerate(state.rosters[9])])

In [184]:
results = pd.concat([tm1, tm2, tm3, tm4, tm5, tm6, tm7, tm8, tm9, tm10], axis= 1)
results.columns = ['tm1', 'tm2', 'tm3', 'tm4', 'tm5', 'tm6', 'tm7', 'tm8', 'tm9', 'tm10']

In [185]:
results

,tm1,tm2,tm3,tm4,tm5,tm6,tm7,tm8,tm9,tm10
0,Pete Alonso,Matt Olson,Bryan Reynolds,Rafael Devers,Shohei Ohtani,Mike Trout,Vladimir Guerrero Jr.,Nolan Arenado,Yordan Alvarez,Aaron Judge
1,Fernando Tatis Jr.,Kevin Gausman,Alex Bregman,Manny Machado,Julio Rodríguez,José Ramírez,Juan Soto,Eloy Jiménez,Freddie Freeman,Paul Goldschmidt
2,Wander Franco,Yu Darvish,J.T. Realmuto,Ronald Acuña Jr.,Corey Seager,Marcus Semien,Austin Riley,Xander Bogaerts,Kyle Tucker,Jacob deGrom
3,Sandy Alcantara,José Abreu,Dylan Cease,Kyle Schwarber,Freddie Freeman,Corbin Burnes,Bryce Harper,Christian Walker,Mookie Betts,Gerrit Cole
4,Aaron Judge,Yordan Alvarez,Shohei Ohtani,Vladimir Guerrero Jr.,Nolan Arenado,Austin Riley,Jacob deGrom,Pete Alonso,Paul Goldschmidt,Rafael Devers
5,Will Smith,Alex Bregman,Trea Turner,Bo Bichette,Corbin Burnes,Corey Seager,Fernando Tatis Jr.,Jose Altuve,Manny Machado,José Ramírez
6,Marcus Semien,Salvador Perez,Matt Olson,Gerrit Cole,Jorge Polanco,Carlos Rodón,Max Scherzer,Shane McClanahan,J.T. Realmuto,Ozzie Albies
7,Yandy Díaz,Xander Bogaerts,Shane Bieber,Justin Verlander,Alejandro Kirk,Vinnie Pasquantino,Willson Contreras,Aaron Nola,Max Muncy,Carlos Correa
8,Edwin Díaz,Gleyber Torres,Jazz Chisholm Jr.,Ketel Marte,Luis Castillo,Adley Rutschman,Luis Arraez,Sean Murphy,Bobby Witt Jr.,Daulton Varsho
9,Bryce Harper,Mike Trout,Andrés Giménez,Francisco Lindor,Zack Wheeler,Willy Adames,Sandy Alcantara,Matt Chapman,Spencer Strider,J.D. Martinez
